In [1]:
import nltk
nltk.download('punkt')
from nltk import *
import numpy as np
from urllib import request
from collections import Counter
import matplotlib.pyplot as plt
from sklearn import svm
from scipy.optimize import curve_fit
from PyDictionary import PyDictionary
from vocabulary.vocabulary import Vocabulary as vb

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Meet\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
url = "http://www.gutenberg.org/files/50169/50169-0.txt"  # Random Corpus used: Hermia Suydam, by Gertrude Atherton
res = request.urlopen(url)
text = res.read().decode('utf8')
# text= text.lower()
# loading the corpus from the Ebook itself hence NOTE that there might be some extra text in the corpus too

In [3]:
sent_tokenize_list = sent_tokenize(text)
word_tokenize_list = word_tokenize(text)
#tweet_tokenizer = TweetTokenizer()
#tweet_tokenizer_list = tweet_tokenizer.tokenize(text)

In [4]:
import string
translate_table_1 = dict((ord(char), None) for char in string.punctuation)   
translate_table_2 = dict((ord(char), " ") for char in string.whitespace[1:])

sent_tokenize_filtered_list_og = [(sent_tokenize_list[i].translate(translate_table_1)).translate(translate_table_2)
                          for i in range(2,len(sent_tokenize_list)-117)]
sent_tokenize_filtered_list_og = [" SsSs "+sent_tokenize_filtered_list_og[i]+" EeEe " for i in range(len(sent_tokenize_filtered_list_og))]

In [5]:
import sklearn
sent_tokenize_filtered_list, sent_tokenize_filtered_list_test = sklearn.model_selection.train_test_split(
    sent_tokenize_filtered_list_og, test_size = 0.2)

In [6]:
len(sent_tokenize_filtered_list)

2050

In [7]:
mod_text = "".join(sent_tokenize_filtered_list)
# tokens = sorted(set(word_tokenize_list),key = word_tokenize_list.index)
tokens = word_tokenize(mod_text)
translate_table_3 = dict((ord(char), None) for char in string.punctuation[0:17] or string.punctuation[18] or string.punctuation[20:])
tokens = [tokens[i].translate(translate_table_3) for i in range(len(tokens))]
for i in range(len(tokens)):
    if tokens[i].isalpha()==False:
        tokens[i]=""
tokens = [value for value in tokens if value != ""]
# for i in range(len(tokens)):
#    if tokens[i].isalpha()==False:
#        tokens[i]=""
# tokens = sorted(set(tokens),key = tokens.index)

In [8]:
from collections import Counter
unigrams = list(nltk.ngrams(tokens, 1))
bigrams = list(nltk.ngrams(tokens, 2))
trigrams = list(nltk.ngrams(tokens, 3))
quadgrams = list(nltk.ngrams(tokens, 4))

In [9]:
count_unigrams = Counter(unigrams)
count_bigrams = Counter(bigrams)
count_trigrams = Counter(trigrams)
count_quadgrams = Counter(quadgrams)
no_of_tokens = len(tokens)

In [10]:
prob_unigrams = dict()
for i in range(len(unigrams)):
    prob_unigrams[unigrams[i]] = count_unigrams[unigrams[i]]/no_of_tokens 
    
prob_bigrams = dict()
for i in range(len(bigrams)):
    prob_bigrams[bigrams[i]] = count_bigrams[bigrams[i]]/count_unigrams[(bigrams[i][0],)] 

prob_trigrams = dict()
for i in range(len(trigrams)):
    prob_trigrams[trigrams[i]] = count_trigrams[trigrams[i]]/count_bigrams[(trigrams[i][0],trigrams[i][1])] 
    
prob_quadgrams = dict()
for i in range(len(quadgrams)):
    prob_quadgrams[quadgrams[i]] = count_quadgrams[quadgrams[i]]/count_trigrams[(quadgrams[i][0],quadgrams[i][1],quadgrams[i][2])] 

In [11]:
def Probability(sentence, model_name):  # in logspace
    
    sentence = ' SsSs '+sentence+' EeEe '
    words = word_tokenize(sentence)
    len_words = len(words)
    
    if model_name==unigrams:
        result_prob = 0
        for i in range(len(words)):
            result_prob = result_prob + prob_unigrams[(words[i],)]
    
    if model_name==bigrams:
        result_prob = 0 #prob_unigrams[(words[0],)]
        for i in range(1,len(words)):
            result_prob = result_prob + prob_bigrams[(words[i-1],words[i])]
    
    if model_name==trigrams:
        result_prob = prob_bigrams[(words[0],words[1])] #prob_unigrams[(words[0],)] + prob_bigrams[(words[0],words[1])]
        for i in range(2,len(words)):
            result_prob = result_prob + prob_trigrams[(words[i-2],words[i-1],words[i])]
            
    if model_name==quadgrams:
        result_prob = prob_bigrams[(words[0],words[1])] + prob_trigrams[(words[0],words[1],words[2])]#prob_unigrams[(words[0],)] + prob_bigrams[(words[0],words[1])] + prob_trigrams[(words[0],words[1],words[2])]
        for i in range(3,len(words)):
            result_prob = result_prob + prob_quadgrams[(words[i-3],words[i-2],words[i-1],words[i])]
    
    return result_prob
        

In [12]:
def Generator(model_name):
    
    import operator
    import random
    prob_unigrams_sorted = dict(sorted(prob_unigrams.items(), key=operator.itemgetter(1))[::-1])
    prob_bigrams_sorted = dict(sorted(prob_bigrams.items(), key=operator.itemgetter(1))[::-1])
    prob_trigrams_sorted = dict(sorted(prob_trigrams.items(), key=operator.itemgetter(1))[::-1])
    prob_quadgrams_sorted = dict(sorted(prob_quadgrams.items(), key=operator.itemgetter(1))[::-1])
    
    if model_name==bigrams:
        sentence=["SsSs"]
        word=sentence[0]
        counter = 0
        while word!="EeEe" and counter<=25:
            candidate_bigrams = []
            candidate_bigrams_keys = list(prob_bigrams_sorted.keys())
            for i in range(len(candidate_bigrams_keys)):
                if candidate_bigrams_keys[i][0]==word:
                    candidate_bigrams = candidate_bigrams + [candidate_bigrams_keys[i]]
            if word==sentence[0]:
                word = random.choice(candidate_bigrams)[1]
            else:
#                result = random.choice(candidate_bigrams)
                maximum = 0.0
                for i in range(len(candidate_bigrams)):
                    if prob_bigrams_sorted[candidate_bigrams[i]]>maximum:
                        result=candidate_bigrams[i]
                word = result[1]
            sentence = sentence + [word]
            counter = counter + 1
            #print(sentence)
        if counter>25:
            sentence = sorted(set(sentence),key = sentence.index)
        return " ".join(sentence)
    
    if model_name==trigrams:
        sentence=["SsSs"]
        word=sentence[0]
        candidate_bigrams = []
        candidate_bigrams_keys = list(prob_bigrams_sorted.keys())
        for i in range(len(candidate_bigrams_keys)):
            if candidate_bigrams_keys[i][0]==word:
                candidate_bigrams = candidate_bigrams + [candidate_bigrams_keys[i]]
        if word==sentence[0]:
            word = random.choice(candidate_bigrams)[1]
        sentence = sentence + [word]
        while word!="EeEe":
            candidate_trigrams = []
            candidate_trigrams_keys = list(prob_trigrams_sorted.keys())
            for i in range(len(candidate_trigrams_keys)):
                if candidate_trigrams_keys[i][0]==sentence[-2] and candidate_trigrams_keys[i][1]==sentence[-1]:
                    candidate_trigrams = candidate_trigrams + [candidate_trigrams_keys[i]]
#            result = random.choice(candidate_trigrams)
            maximum = 0.0
            for i in range(len(candidate_trigrams)):
                if prob_trigrams_sorted[candidate_trigrams[i]]>maximum:
                    result=candidate_trigrams[i]
            word = result[2]
            sentence = sentence + [word]
        return " ".join(sentence)
    
    if model_name==quadgrams:
        sentence=["SsSs"]
        word=sentence[0]
        candidate_trigrams = []
        candidate_trigrams_keys = list(prob_trigrams_sorted.keys())
        for i in range(len(candidate_trigrams_keys)):
            if candidate_trigrams_keys[i][0]==word:
                candidate_trigrams = candidate_trigrams + [candidate_trigrams_keys[i]]
        if word==sentence[0]:
            trigram = random.choice(candidate_trigrams)
        sentence = sentence + [trigram[1]] + [trigram[2]]
        while word!="EeEe":
            candidate_quadgrams = []
            candidate_quadgrams_keys = list(prob_quadgrams_sorted.keys())
            for i in range(len(candidate_quadgrams_keys)):
                if candidate_quadgrams_keys[i][0]==sentence[-3] and candidate_quadgrams_keys[i][1]==sentence[-2] and candidate_quadgrams_keys[i][2]==sentence[-1]:
                    candidate_quadgrams = candidate_quadgrams + [candidate_quadgrams_keys[i]]
#            result = random.choice(candidate_quadgrams) 
            maximum = 0.0
            for i in range(len(candidate_quadgrams)):
                if prob_quadgrams_sorted[candidate_quadgrams[i]]>maximum:
                    result=candidate_quadgrams[i]
            word = result[3]
            sentence = sentence + [word]
        return " ".join(sentence)
    
    if model_name==unigrams:
        sentence=["SsSs"]
        word=sentence[0]
        while word!="EeEe":
            candidate_unigrams_keys = list(prob_unigrams_sorted.keys())
            result = random.choice(candidate_unigrams_keys)
            word=result[0]
            sentence=sentence + [word]
        return " ".join(sentence)
        

In [13]:
Generator(quadgrams)

'SsSs She died before the year was out not of indigestion however but of relaxation from the terrible strain to which her delicate constitution had been subjected during the ten preceding years EeEe'

In [14]:
uniq_tokens = sorted(set(tokens),key = tokens.index)
voc_len = len(uniq_tokens)
count_bigrams_smoothed = dict() 
for (first,second) in prob_bigrams:
    count_bigrams_smoothed[(first,second)] = ((count_bigrams[(first,second)] + 1) * count_unigrams[(first,)])/(count_unigrams[(first,)] + voc_len)

In [15]:
count_bigrams_diff=dict()
for k, v in count_bigrams.items():
    count_bigrams_diff[k] = v - count_bigrams_smoothed.get(k, 0)

In [16]:
count_bigrams_diff

{('SsSs', 'But'): 36.16148246194573,
 ('But', 'he'): 4.938511326860842,
 ('he', 'is'): 12.24682021313166,
 ('is', 'clever'): 0.8979486295466299,
 ('clever', 'there'): 0.9942039485600435,
 ('there', 'is'): 6.923713566030947,
 ('is', 'no'): 1.8469229443199449,
 ('no', 'denying'): 1.948580610605249,
 ('denying', 'that'): 1.9989104775739968,
 ('that', 'Oh'): 0.8676844783715013,
 ('Oh', 'horribly'): 0.9927601809954751,
 ('horribly', 'clever'): 0.9996367598982928,
 ('clever', 'EeEe'): 1.9913059228400651,
 ('EeEe', 'SsSs'): 1492.745863666446,
 ('SsSs', 'The'): 87.89609530112509,
 ('The', 'great'): 0.9535131298793471,
 ('great', 'mystery'): 0.9877234157790215,
 ('mystery', 'of'): 1.9972776769509981,
 ('of', 'life'): 9.07934086054318,
 ('life', 'had'): 3.9213302342213483,
 ('had', 'poured'): 0.851664984863774,
 ('poured', 'through'): 0.9989106753812637,
 ('through', 'flooding'): 0.981284865934857,
 ('flooding', 'them'): 0.9996367598982928,
 ('them', 'with'): 1.9687219126370663,
 ('with', 'light

In [17]:
count_bigrams_values = np.array(list(count_bigrams.values()))
good_turing_N = dict()
good_turing_count_smoothed = dict()
num = 11
for i in range(1,num+1):
    good_turing_N[i] = np.where(count_bigrams_values==i)[0].shape[0]

for i in range(1,num):
    good_turing_count_smoothed[i] = ((i + 1) * good_turing_N[i+1])/good_turing_N[i]
    

In [18]:
good_turing_count_diff=dict()
for k, v in good_turing_count_smoothed.items():
    good_turing_count_diff[k] = k - v

In [19]:
good_turing_count_smoothed

{1: 0.24651313655530327,
 2: 1.043421052631579,
 3: 1.9470365699873897,
 4: 2.5259067357512954,
 5: 4.061538461538461,
 6: 5.462121212121212,
 7: 4.737864077669903,
 8: 7.819672131147541,
 9: 6.60377358490566,
 10: 8.8}

In [20]:
good_turing_N

{1: 18498,
 2: 2280,
 3: 793,
 4: 386,
 5: 195,
 6: 132,
 7: 103,
 8: 61,
 9: 53,
 10: 35,
 11: 28}

In [21]:
good_turing_count_diff

{1: 0.7534868634446967,
 2: 0.956578947368421,
 3: 1.0529634300126103,
 4: 1.4740932642487046,
 5: 0.9384615384615387,
 6: 0.5378787878787881,
 7: 2.2621359223300974,
 8: 0.18032786885245944,
 9: 2.3962264150943398,
 10: 1.1999999999999993}

In [22]:
# uniq_tokens = sorted(set(tokens),key = tokens.index)
# voc_len = len(uniq_tokens)
# count_bigrams_smoothed = dict() 
# for i in range(len(uniq_tokens)):
#     for j in range(len(uniq_tokens)):
#         if (uniq_tokens[i],uniq_tokens[j]) in prob_bigrams:
#             count_bigrams_smoothed[(uniq_tokens[i],uniq_tokens[j])] = ((count_bigrams[(uniq_tokens[i],uniq_tokens[j])] + 1) * count_unigrams[(uniq_tokens[i],)])/(count_unigrams[(uniq_tokens[i],)] + voc_len)
#         else:
#             count_bigrams_smoothed[(uniq_tokens[i],uniq_tokens[j])] = count_unigrams[(uniq_tokens[i],)]/(count_unigrams[(uniq_tokens[i],)] + voc_len)
             

In [23]:
# mod_text_og = "".join(sent_tokenize_filtered_list_og)
# # tokens = sorted(set(word_tokenize_list),key = word_tokenize_list.index)
# tokens_og = word_tokenize(mod_text_og)
# uniq_tokens = sorted(set(tokens_og),key = tokens_og.index)
# voc_len = len(uniq_tokens)
# prob_bigrams_add1_smoothed = dict() 
# for i in range(len(uniq_tokens)):
#     for j in range(len(uniq_tokens)):
#         if (uniq_tokens[i],uniq_tokens[j]) in prob_bigrams:
#             prob_bigrams_add1_smoothed[(uniq_tokens[i],uniq_tokens[j])] = (count_bigrams[(uniq_tokens[i],uniq_tokens[j])] + 1)/(count_unigrams[(uniq_tokens[i],)] + voc_len)
#         else:
#             prob_bigrams_add1_smoothed[(uniq_tokens[i],uniq_tokens[j])] = 1/(count_unigrams[(uniq_tokens[i],)] + voc_len)
            

In [24]:
mod_text_og = "".join(sent_tokenize_filtered_list_og)
# tokens = sorted(set(word_tokenize_list),key = word_tokenize_list.index)
tokens_og = word_tokenize(mod_text_og)
uniq_tokens = sorted(set(tokens_og),key = tokens_og.index)
voc_len = len(uniq_tokens)

In [25]:
#result_prob_add1_smoothed = 1
perplexity_add1 = []
for i in range(len(sent_tokenize_filtered_list_test)):
    words = word_tokenize(sent_tokenize_filtered_list_test[i])
    words = [words[k].translate(translate_table_3) for k in range(len(words))]
    for z in range(len(words)):
        if words[z].isalpha()==False:
            words[z]=""
    words = [value for value in words if value != ""]
    res_prob = 1
    for j in range(len(words)-1):
        if (words[j],words[j+1]) in prob_bigrams:
            int_prob = (count_bigrams[(words[j],words[j+1])] + 1)/(count_unigrams[(words[j],)] + voc_len)
        else:
            int_prob = 1/(count_unigrams[(words[j],)] + voc_len)
        res_prob = res_prob * int_prob
        print(res_prob,int_prob)
    perp = ((1.0/res_prob)**(1.0/len(words)))
#     result_prob_add1_smoothed = result_prob_add1_smoothed * probab
    perplexity_add1 = perplexity_add1 + [perp] 

0.01869708372530574 0.01869708372530574
2.7058008285536525e-06 0.0001447178002894356
4.1911413081686066e-10 0.00015489467162329616
4.396521677982953e-13 0.0010490034467256107
1.933386841681158e-16 0.00043975373790677223
7.898897623646294e-20 0.0004085523627944982
1.2234971536007272e-23 0.00015489467162329616
2.896994049567626e-26 0.0023677979479084454
2.7831746676373807e-28 0.009607112130771436
3.7902419551101464e-32 0.00013618412093149938
5.8717923394425194e-36 0.00015491866769945777
9.078219448736116e-40 0.00015460729746444033
1.3979395516994327e-43 0.0001539882968894364
6.343697859776581e-47 0.00045378913931326575
1.946488238680716e-49 0.0030683810637054353
2.816914961911311e-53 0.0001447178002894356
4.3639271292196915e-57 0.00015491866769945777
6.67063150293441e-61 0.0001528584530724549
1.0335654637332523e-64 0.0001549426712116517
3.198407747898042e-68 0.00030945381401825775
1.9083578448079012e-71 0.0005966587112171838
2.835177306206955e-75 0.0001485663348685188
4.3888193594534914e

0.0010583254938852304 0.0010583254938852304
1.6377677095097962e-07 0.00015475085112968121
1.0914085762427004e-10 0.0006664001066240171
1.687397304023965e-14 0.00015460729746444033
2.6076298934074565e-18 0.00015453562046051616
1.5452621590562704e-21 0.0005925925925925926
2.257834832051827e-25 0.0001461133839859731
3.497807640668981e-29 0.00015491866769945777
1.6064005460850134e-31 0.0045925925925925926
2.0102622276123304e-35 0.00012514078338130396
3.114753993821398e-39 0.0001549426712116517
4.826083039698479e-43 0.0001549426712116517
6.432204504462854e-47 0.0001332800213248034
5.974183750275096e-50 0.0009287925696594427
7.962393376349588e-54 0.0001332800213248034
2.4659007049704516e-57 0.0003096934035305048
0.032220131702728125 0.032220131702728125
3.821513026268718e-05 0.0011860637509266123
5.896486693826134e-09 0.00015429717636167257
8.958503029210169e-13 0.00015192950470981463
2.773101076988135e-16 0.00030954960532425323
7.609002817912293e-20 0.00027438606118809165
1.1785939928612598

1.0590756657449004e-38 0.0003097893432465923
0.003410159924741298 0.003410159924741298
4.2048827678684316e-06 0.0012330456226880395
1.868836785719303e-09 0.00044444444444444447
5.717720011379236e-13 0.0003059507419305492
2.765827622039208e-15 0.004837291116974494
4.137982678095763e-19 0.00014961101137043686
6.40752969664875e-23 0.0001548467017652524
1.979159751860618e-26 0.0003088803088803089
5.143757851696929e-28 0.025989604158336666
6.436938870850868e-32 0.00012514078338130396
9.972019939350685e-36 0.00015491866769945777
1.4298852795168748e-39 0.00014338973329509606
6.612695912672459e-43 0.0004624633883150917
2.6095879686947355e-46 0.0003946329913180742
7.946370184819535e-49 0.0030450669914738123
2.3992663601508257e-52 0.00030193236714975844
2.2287657781243155e-55 0.0009289363678588017
9.173133947555147e-59 0.00041157909178213747
1.4206495195222468e-62 0.00015487068297971194
3.675928213389989e-65 0.0025874982976984884
5.628430888669406e-69 0.0001531159087429184
8.624625940345396e-73 

2.0631484509903386e-32 0.0003096934035305048
5.163680267777095e-36 0.0002502815667626079
7.99950467510007e-40 0.00015491866769945777
2.467079313831941e-43 0.00030840400925212025
1.0629223013092599e-45 0.004308423711187045
8.371740015743763e-48 0.007876154263986964
0.007173095014111007 0.007173095014111007
1.390964128631311e-05 0.0019391408114558472
8.061223579433852e-09 0.0005795421616922632
3.744181876188505e-12 0.00046446818392940084
1.1439602432595493e-15 0.00030553009471432935
5.2156242093900426e-18 0.004559270516717325
1.3555200864068485e-19 0.025989604158336666
1.6963084550204585e-23 0.00012514078338130396
2.6283056321977977e-27 0.0001549426712116517
8.139689167537311e-31 0.0003096934035305048
1.1167086249879698e-34 0.00013719303059404582
1.7278487157480577e-38 0.00015472690700912888
2.6484499015144965e-42 0.00015328019619865113
4.101037320400273e-46 0.0001548467017652524
1.26438640986597e-49 0.0003083089255433945
1.93983800224911e-53 0.00015342129487572875
1.4942520430204207e-56

5.370731191963996e-29 0.00015356265356265356
7.681251704753998e-33 0.00014302059496567506
1.190153657383638e-36 0.0001549426712116517
2.2014600201102564e-38 0.01849727559506739
1.3774621574960932e-41 0.0006257039169065198
6.358495880117371e-45 0.0004616094783812894
2.787184050314453e-48 0.00043834015195791935
8.347361636161885e-52 0.0002994908655286014
1.2927615976710369e-55 0.00015487068297971194
1.9152023669200545e-59 0.00014814814814814815
5.923000980114595e-63 0.0003092624091541673
1.0063795301399528e-65 0.001699104108742663
1.38068257667712e-68 0.0013719303059404582
4.260051146797655e-72 0.000308546744831842
5.784968966319467e-76 0.00013579576317218902
1.7915667284978218e-79 0.0003096934035305048
2.4579046899407624e-83 0.00013719303059404582
3.808343182430683e-87 0.0001549426712116517
0.014346190028222013 0.014346190028222013
2.1786165570572535e-06 0.00015186028853454822
6.723087662574459e-10 0.00030859435272334514
3.5842130681457864e-13 0.0005331200852992136
5.5474587032127945e-1

...

5.2545804111194865e-05 0.0016308376575240918
1.590610083583922e-08 0.0003027092477675193
4.909290381431858e-12 0.00030864197530864197
6.457893161578345e-16 0.00013154433043935807
9.99983456422785e-20 0.0001548467017652524
3.946264626767107e-23 0.0003946329913180742
3.052022139804414e-26 0.0007733952049497294
1.3824179768254943e-28 0.004529514903565166
3.459937371606793e-32 0.0002502815667626079
5.3575989030764836e-36 0.0001548467017652524
8.239924489505512e-40 0.00015379883112888343
3.8283682001729184e-43 0.00046461204893913583
0.009877704609595485 0.009877704609595485
2.5644621008418334e-05 0.002596212583995113
3.836717685281019e-09 0.00014961101137043686
1.121191608790479e-12 0.0002922267679719462
1.7369351026963266e-16 0.00015491866769945777
2.6908367199013577e-20 0.00015491866769945777
4.164737223187367e-24 0.00015477480266212662
6.4529551025524745e-28 0.0001549426712116517
7.961696610181955e-31 0.0012338062924120913
0.01869708372530574 0.01869708372530574
1.8940605799875567e-05 0.

5.525835191224877e-107 0.000154273372415921
2.75332268323148e-109 0.004982636267552468
0.009877704609595485 0.009877704609595485
9.051042708853529e-06 0.0009163103237629811
2.09127604178686e-08 0.0023105360443622922
6.3142392566028375e-12 0.00030193236714975844
9.659230926423188e-16 0.0001529753709652746
1.4966270415902059e-19 0.0001549426712116517
0.01869708372530574 0.01869708372530574
1.082320331421461e-05 0.0005788712011577424
1.6743817008376563e-09 0.0001547029702970297
1.2842580339275801e-11 0.007670043415340087
1.973656114841832e-15 0.0001536806516059628
3.053777061491307e-19 0.00015472690700912888
4.7301379515045027e-23 0.00015489467162329616
1.6760618222862973e-25 0.0035433677399476196
5.175426346414381e-29 0.0003087849312953528
2.4015899519324275e-32 0.0004640371229698376
6.522514806986495e-36 0.00027159152634437803
4.000929186926235e-39 0.0006134028523232633
1.0306985327738443e-41 0.002576147901197151
4.2109432089357663e-45 0.0004085523627944982
1.304303301513324e-48 0.00030

3.7700607496079757e-209 0.00013154433043935807
5.841432831744617e-213 0.0001549426712116517
8.91821806373224e-217 0.00015267175572519084
5.483072895009063e-220 0.0006148170919151552
8.423832992793153e-224 0.00015363343063450608
5.120871120238999e-227 0.0006079027355623101
7.896485921725518e-231 0.00015420200462606013
1.5873560690031197e-233 0.0020102056595020876
4.8916982095627724e-237 0.0003081664098613251
1.5149266675635714e-240 0.0003096934035305048
5.978400424481339e-244 0.0003946329913180742
5.537264980995374e-247 0.0009262117937635072
7.519371239809036e-250 0.0013579576317218902
5.1200948112549606e-253 0.0006809206046574969
2.37812113852994e-256 0.00046446818392940084
3.6807322992260334e-260 0.00015477480266212662
0.032220131702728125 0.032220131702728125
0.00024362145542463075 0.007561156412157153
1.0599918292587269e-07 0.00043509789702683103
4.919468968278355e-10 0.004641044234952864
6.69953556894778e-14 0.00013618412093149938
1.0374009862105575e-17 0.0001548467017652524
1.6046

6.147066496673909e-46 0.00015489467162329616
0.009289746001881467 0.009289746001881467
1.4204504590032824e-06 0.0001529051987767584
2.200542926418718e-10 0.00015491866769945777
6.813881177949273e-14 0.0003096454559529339
8.963274372466816e-18 0.00013154433043935807
1.3885785240072526e-21 0.00015491866769945777
1.2427612088370398e-24 0.0008949880668257757
7.692734192739337e-28 0.0006190034045187249
2.378705687303444e-31 0.00030921459492888067
0.0018814675446848542 0.0018814675446848542
8.622674356942503e-07 0.00045829514207149406
1.3261572373027537e-10 0.00015379883112888343
3.6017306825169844e-14 0.00027159152634437803
5.5737088865939095e-18 0.00015475085112968121
1.0309842936200377e-19 0.01849727559506739
1.290181821574318e-23 0.00012514078338130396
3.367586514166038e-26 0.0026101642868109932
0.014346190028222013 0.014346190028222013
2.1786165570572535e-06 0.00015186028853454822
3.3584346493868563e-10 0.00015415446277169725
5.2028422143870736e-14 0.00015491866769945777
7.9896225650907

1.3055856610787654e-56 0.0021324803411968546
1.9956980450607848e-60 0.0001528584530724549
3.091708822712292e-64 0.00015491866769945777
0.00023518344308560678 0.00023518344308560678
7.286861133558692e-08 0.00030983733539891554
3.230468952368269e-10 0.004433279148810403
9.360964799676236e-14 0.0002897710808461316
4.2900846927938754e-17 0.00045829514207149406
1.9649242867758208e-20 0.0004580152671755725
3.0430916629639474e-24 0.00015487068297971194
9.427173677087818e-28 0.0003097893432465923
1.034674021356874e-29 0.010975442447523666
2.5895983515376646e-33 0.0002502815667626079
8.023542529938542e-37 0.00030983733539891554
6.416267516944056e-40 0.0007996801279488205
9.889438219704156e-44 0.00015413070283600493
1.522857748645543e-47 0.0001539882968894364
2.3591909351596328e-51 0.00015491866769945777
3.653129351439506e-55 0.0001548467017652524
5.658502712886471e-59 0.00015489467162329616
1.0466688413569754e-60 0.01849727559506739
3.92942876244641e-64 0.0003754223501439119
6.0855331616019975e

In [26]:
import statistics as sts
perplexity_add1_mean = sts.mean(perplexity_add1)

In [27]:
perplexity_add1_mean

1411.7486676598023

In [28]:
count_bigrams_values = np.array(list(count_bigrams.values()))
good_turing_N = dict()
good_turing_count_smoothed = dict()
for i in set(count_bigrams_values):
    good_turing_N[i] = np.where(count_bigrams_values==i)[0].shape[0]

# # for i in range(1,11):
# #     good_turing_count_smoothed[i] = ((i + 1) * good_turing_N[i+1])/good_turing_N[i]

In [29]:
good_turing_N

{128: 1,
 1: 18498,
 2: 2280,
 2049: 1,
 4: 386,
 5: 195,
 3: 793,
 7: 103,
 8: 61,
 6: 132,
 10: 35,
 11: 28,
 12: 23,
 13: 14,
 14: 20,
 15: 19,
 16: 10,
 273: 1,
 18: 11,
 17: 10,
 20: 3,
 19: 10,
 21: 4,
 23: 4,
 24: 3,
 25: 7,
 22: 11,
 27: 5,
 28: 11,
 29: 5,
 158: 1,
 30: 3,
 31: 3,
 32: 3,
 34: 2,
 35: 6,
 36: 1,
 38: 3,
 40: 2,
 42: 2,
 44: 2,
 9: 53,
 47: 1,
 48: 1,
 46: 2,
 50: 2,
 52: 3,
 53: 1,
 54: 1,
 55: 1,
 56: 1,
 57: 1,
 58: 1,
 60: 2,
 62: 1,
 194: 1,
 66: 1,
 78: 1,
 79: 1,
 26: 1,
 83: 1,
 87: 1,
 108: 1,
 112: 1,
 121: 1}

In [30]:
d = 1
i = 10
while (d >= 1.0 or d < 0.0):
    d = sts.median(list(good_turing_count_diff.values())[:i])
    i = i-1

In [31]:
d

0.956578947368421

In [32]:
mod_text_og = "".join(sent_tokenize_filtered_list_og)
# tokens = sorted(set(word_tokenize_list),key = word_tokenize_list.index)
tokens_og = word_tokenize(mod_text_og)
translate_table_3 = dict((ord(char), None) for char in string.punctuation[0:17] or string.punctuation[18] or string.punctuation[20:])
tokens_og = [tokens_og[i].translate(translate_table_3) for i in range(len(tokens_og))]
for i in range(len(tokens_og)):
    if tokens_og[i].isalpha()==False:
        tokens_og[i]=""
tokens_og = [value for value in tokens_og if value != ""]
# for i in range(len(tokens)):
#    if tokens[i].isalpha()==False:
#        tokens[i]=""
# tokens = sorted(set(tokens),key = tokens.index)

from collections import Counter
unigrams_og = list(nltk.ngrams(tokens_og, 1))
bigrams_og = list(nltk.ngrams(tokens_og, 2))
trigrams_og = list(nltk.ngrams(tokens_og, 3))
quadgrams_og = list(nltk.ngrams(tokens_og, 4))

count_unigrams_og = Counter(unigrams_og)
count_bigrams_og = Counter(bigrams_og)
count_trigrams_og = Counter(trigrams_og)
count_quadgrams_og = Counter(quadgrams_og)
no_of_tokens_og = len(tokens_og)

prob_unigrams_og = dict()
for i in range(len(unigrams_og)):
    prob_unigrams_og[unigrams_og[i]] = count_unigrams_og[unigrams_og[i]]/no_of_tokens_og 
    
prob_bigrams_og = dict()
for i in range(len(bigrams_og)):
    prob_bigrams_og[bigrams_og[i]] = count_bigrams_og[bigrams_og[i]]/count_unigrams_og[(bigrams_og[i][0],)] 

prob_trigrams_og = dict()
for i in range(len(trigrams_og)):
    prob_trigrams_og[trigrams_og[i]] = count_trigrams_og[trigrams_og[i]]/count_bigrams_og[(trigrams_og[i][0],trigrams_og[i][1])] 
    
prob_quadgrams_og = dict()
for i in range(len(quadgrams_og)):
    prob_quadgrams_og[quadgrams_og[i]] = count_quadgrams_og[quadgrams_og[i]]/count_trigrams_og[(quadgrams_og[i][0],quadgrams_og[i][1],quadgrams_og[i][2])] 



In [33]:
#result_prob_add1_smoothed = 1
perplexity_good_turing = []
for i in range(len(sent_tokenize_filtered_list_test)):
    words = word_tokenize(sent_tokenize_filtered_list_test[i])
    words = [words[k].translate(translate_table_3) for k in range(len(words))]
    for z in range(len(words)):
        if words[z].isalpha()==False:
            words[z]=""
    words = [value for value in words if value != ""]
    res_prob = 1
    for j in range(len(words)-1):
        if (words[j],words[j+1]) in prob_bigrams:
            int_prob = (count_bigrams[(words[j],words[j+1])] - d)/(count_unigrams[(words[j],)])
        else:
            int_prob = (d/count_unigrams_og[(words[j],)])*prob_unigrams_og[(words[j+1],)]
        res_prob = res_prob * int_prob
        print(res_prob,int_prob)
    perp = ((1.0/res_prob)**(1.0/len(words)))
#     result_prob_add1_smoothed = result_prob_add1_smoothed * probab
    perplexity_good_turing = perplexity_good_turing + [perp] 

0.07660654685494223 0.07660654685494223
1.0237840496918076e-08 1.3364184808256486e-07
1.3490490694412985e-11 0.0013177086220940892
3.1067682547275283e-13 0.023029319875030044
8.808878811495031e-16 0.002835383295194508
1.033899842744e-18 0.0011737019714640935
7.272698258692258e-21 0.007034238673825801
1.0163672504611534e-22 0.013975105446543188
1.2713077503048126e-23 0.1250835020242915
6.569463466320357e-31 5.1674847925258404e-08
8.486900639316137e-35 0.00012918711981314604
4.6871025176111365e-39 5.522749372011992e-05
3.2665847704321395e-43 6.969305147814457e-05
2.1709702673086548e-45 0.0066459939658062355
1.0600692536693038e-46 0.04882928475033738
1.7708701769483875e-53 1.6705231010320606e-07
7.343633128906965e-57 0.0004146906546001987
5.319828857315285e-63 7.244137559615664e-07
3.0926401558893966e-67 5.8134203915915704e-05
1.4920632331045361e-69 0.004824561403508781
1.2195653689533392e-71 0.008173684210526316
5.401787569335039e-78 4.4292726793078634e-07
1.308452583609091e-83 2.4222584

2.7234242440975967e-07 0.10374773139745917
6.259957745372636e-10 0.0022985613640400214
2.5162306782294606e-12 0.004019564956471886
1.343242124404723e-19 5.338310736080413e-08
1.7700017288880848e-22 0.0013177086220940892
2.9939232560313775e-23 0.16914804133621725
1.5155812005810728e-30 5.062191215248668e-08
6.044147510407598e-33 0.003988006388631817
4.552921332917519e-40 7.532776665489564e-08
5.653133431289786e-42 0.012416497053040996
0.003435815147625161 0.003435815147625161
7.711111942230843e-06 0.0022443325996629277
2.2371935580728438e-08 0.002901259344739351
5.355307444297987e-13 2.393761337714176e-05
1.779008771424232e-16 0.00033219545094808977
1.2281297217380124e-18 0.006903449502133713
9.635134099691556e-26 7.84537164857162e-08
3.360785103051971e-28 0.0034880522349549424
3.2975911086266964e-29 0.09811966571834993
3.3386073483144358e-37 1.0124382430497337e-08
6.469576012736201e-42 1.9378067971971903e-05
1.597186838651927e-43 0.024687658596292204
1.700571160622855e-50 1.06472900944

4.996716195867223e-09 1.0198983143143106e-06
1.1280311258339564e-10 0.022575449187347267
2.1126933448419952e-12 0.018729034123770968
7.55332426023019e-14 0.035752108930863745
1.2696634132401217e-15 0.016809332811582863
8.407793367879372e-16 0.6622064777327935
2.9546309317083613e-22 3.51415740424361e-07
2.2247672307531428e-25 0.0007529763554823367
2.075861276952434e-27 0.00933068973804374
2.707496698393553e-28 0.13042763157894738
5.8781178320386456e-30 0.021710526315789513
0.01319191270860077 0.01319191270860077
0.0023448318525775285 0.17774767801857586
8.26569905358277e-06 0.0035250711237553344
4.324160887257957e-09 0.000523145212428663
3.647737693545373e-10 0.08435712242562929
1.1664374417400371e-16 3.197700985473911e-07
1.1753718096167555e-20 0.00010076595345425388
2.4302800574782583e-23 0.0020676691729323345
4.472350585429829e-24 0.18402614018363353
1.3583936307045072e-31 3.037314729149201e-08
5.571711002962794e-34 0.004101691054067386
7.481316156797255e-40 1.3427322689240372e-06
2.

5.16490381841935e-35 0.00041447534273384345
7.080099673648684e-36 0.1370809587663426
0.05221630295250321 0.05221630295250321
1.0118510678586841e-07 1.93780679719719e-06
1.9975363497424393e-11 0.00019741406746446373
1.654183714855633e-13 0.008281119465329992
1.7630186455530792e-18 1.0657937384584545e-05
1.2423234599775056e-22 7.046570171626146e-05
8.738799727574849e-26 0.00070342386738258
8.163824971079648e-33 9.342043788140725e-08
8.862046843606212e-35 0.010855263157894757
2.5035745115142754e-39 2.825052220662266e-05
1.0188031781974197e-42 0.00040693942741140996
1.4270163361848434e-45 0.0014006791171477105
1.3931914667303384e-46 0.09762967889036671
8.580133623835353e-48 0.06158617698091382
0.028801668806161747 0.028801668806161747
0.0012722758225795555 0.044173684210526316
5.803114287083088e-06 0.004561207706766918
1.2110183235939183e-06 0.2086842105263158
5.715618518106319e-10 0.0004719679633867286
1.2721071165037592e-10 0.22256683375104427
2.341009625503186e-11 0.18402614018363353
7.

4.946988157149231e-83 0.18402614018363353
1.3975499878944365e-87 2.825052220662266e-05
3.47053506123375e-89 0.02483299410608199
0.13270410783055198 0.13270410783055198
4.918263538771114e-08 3.706187863491894e-07
3.785027393432715e-11 0.0007695861280297414
7.915091881911668e-18 2.091158414241572e-07
7.423552723017026e-20 0.009378984898434401
9.907307800285194e-28 1.3345776840201033e-08
1.612669665374661e-30 0.0016277577096456399
2.5409569635373465e-34 0.00015756214791496202
7.209200110290195e-42 2.837198824593251e-08
8.870973479807671e-44 0.012305073162202158
2.1673054210421814e-44 0.24431427125506072
4.291491832063971e-47 0.0019801047837550937
9.950739832826693e-49 0.02318713450292398
2.662259341238721e-51 0.0026754385964912284
5.160629597091045e-55 0.00019384398496240636
5.143013028545155e-58 0.0009965863528442693
4.152569001995212e-64 8.074194988321626e-07
1.3869916707069054e-66 0.0033400809716599253
6.271574837347738e-67 0.452171052631579
6.041307434715841e-69 0.009632839583989916
9

...

5.600217412535354e-35 0.017594938876142864
5.065986146207978e-38 0.0009046052631578964
1.198629613898713e-40 0.0023660341329514263
3.9830425906037524e-43 0.0033229969829031178
2.796918585281076e-45 0.007022065472960752
9.686424483416722e-46 0.3463248638838476
8.170726482781236e-48 0.008435234793570754
1.591285358429404e-53 1.947544519796171e-06
4.064428547072014e-56 0.002554179566563472
0.13270410783055198 0.13270410783055198
0.004124051967441597 0.03107704829083017
1.170396585187246e-06 0.0002837977296181636
1.9546096817398375e-09 0.0016700404858299627
2.3477206159976612e-16 1.201119915618093e-07
7.339745401061724e-19 0.0031263282994781335
6.67111922684984e-22 0.0009089033559508529
1.845729518146373e-22 0.27667464114832535
1.3745205011486132e-23 0.07447031039136302
3.8830921941157e-28 2.825052220662266e-05
7.023036951374039e-32 0.00018086196773840446
1.4581380794292476e-37 2.076221568425561e-06
5.071506565734182e-38 0.3478070175438597
0.04002118100128369 0.04002118100128369
0.00640484

8.286230763510827e-65 0.0014705800472063122
2.3169750616126808e-66 0.027961749168460184
3.1867373718929826e-67 0.13753869969040247
6.016151789042827e-71 0.00018878718535469141
3.2193250736601805e-72 0.05351136717533479
1.1509766071735973e-73 0.035752108930863745
3.2183319188553456e-75 0.027961749168460184
9.678286316505122e-76 0.3007236842105263
3.377434012052703e-78 0.0034897025171624716
2.710835193884406e-79 0.08026315789473684
2.8253024272792504e-83 0.00010422258179520024
5.189766440890638e-84 0.18368888196823419
0.04002118100128369 0.04002118100128369
0.0017215158065081858 0.043015117581187014
0.0005909067240275501 0.34324792243767316
1.5092818802561293e-07 0.0002554179566563472
2.7080524874508174e-14 1.7942655529603612e-07
1.3449810291965365e-15 0.04966598821216398
0.07660654685494223 0.07660654685494223
0.00034328822502806234 0.0044811865189289014
2.572208189314266e-07 0.0007492852949162468
2.879261088644631e-08 0.11193732686980609
1.24913097951066e-13 4.338373426560874e-06
4.841

1.1185879668336117e-175 1.9378067971971906e-05
2.1676073653931585e-180 1.9378067971971906e-05
2.261727158890493e-181 0.1043421052631579
6.324184750439006e-183 0.027961749168460184
2.953023661718439e-188 4.669413969149856e-06
1.1015599448627094e-191 0.0003730278084604591
2.772221232045611e-198 2.516632204152195e-07
3.0093191005758335e-200 0.010855263157894757
8.032340231447913e-208 2.6691553680402066e-08
1.1300279659475558e-209 0.014068477347651601
1.5081101056722374e-217 1.3345776840201033e-08
3.477686956295187e-220 0.0023059900886646563
3.5677511059938466e-225 1.0258977161632184e-05
1.402003407795659e-226 0.03929655870445344
7.029211310018546e-231 5.013690602272095e-05
1.1399712995466586e-232 0.01621762740183793
9.467331998019554e-238 8.304886273702243e-06
8.042441038398594e-238 0.8494939271255061
9.700312655963236e-241 0.0012061403508771952
1.0529944659433795e-242 0.010855263157894757
9.570702038936136e-246 0.0009089033559508529
1.6124324213624316e-246 0.1684758771929825
1.4252168004

1.551505912922878e-14 2.825052220662266e-05
1.100385845647012e-17 0.0007092372877741716
4.101283832872933e-18 0.3727132486388385
1.0789341179346749e-20 0.0026307228709379175
4.2589504655316194e-23 0.003947368421052639
1.5000635866241826e-25 0.00352214377406932
1.4803259078528143e-28 0.0009868421052631597
2.63124492831578e-29 0.17774767801857586
3.7951126424321363e-31 0.014423258745666563
1.5872363470433056e-37 4.182316828483144e-07
1.3138610283695174e-39 0.008277664702027331
0.037582156611039794 0.037582156611039794
1.2889727174195086e-08 3.4297465437118413e-07
9.991120372869028e-13 7.751227188788761e-05
8.67649927117575e-15 0.008684210526315805
2.3158924605455693e-22 2.6691553680402066e-08
7.068209638806114e-25 0.0030520457055855745
1.1431899063184839e-26 0.01617368421052632
2.9200229021588906e-27 0.2554276315789474
9.056462008575523e-30 0.003101503759398502
0.006850449293966624 0.006850449293966624
7.769459797075991e-05 0.011341533180778032
2.9233174143671072e-08 0.000376257486455787

3.1385211128084e-20 0.10144736842105263
1.4064298500090737e-22 0.0044811865189289014
1.4674985145489416e-23 0.1043421052631579
1.237939518512342e-24 0.08435712242562929
8.550490503536673e-30 6.9070341286236486e-06
1.2273480457128438e-35 1.435412442368289e-06
2.2891747585524475e-39 0.00018651390423022956
7.42499665837918e-41 0.03243525480367586
7.517350571459561e-49 1.0124382430497337e-08
1.8558578445683574e-50 0.024687658596292204
2.4845179337562186e-52 0.013387436656464803
8.999094840731063e-59 3.622068779807832e-07
2.2347446913994734e-60 0.02483299410608199
2.1181001283697086e-05 2.1181001283697086e-05
9.197013715289541e-07 0.04342105263157903
8.240119106859172e-08 0.08959559441735329
7.986487621192234e-12 9.692199248120318e-05
9.057901435362474e-14 0.011341533180778032
9.845005260748962e-16 0.010868969298245615
1.9077718112721434e-20 1.9378067971971903e-05
4.1418730113145295e-22 0.021710526315789513
3.8712088547131845e-23 0.09346517491333123
1.093636717165491e-27 2.825052220662266e-

In [34]:
import statistics as sts
perplexity_good_turing_mean = sts.mean(perplexity_good_turing)

In [35]:
perplexity_good_turing_mean

1457.2252123271548

For Hermia Suydam, by Gertrude Atherton; both smoothing methods seem almost comparable